In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import panel as pn
from datetime import datetime, timedelta

from data_sources.ads_analytics import (
    get_facebook_ads_campaign_metrics,
    get_google_ads_campaign_metrics,
)
from data_sources.google_analytics import get_landing_page_report
from data_sources.hubspot_conversions import (
    get_hubspot_conversions,
    get_first_calls,
    get_first_call_verbal_agreements,
    get_placement_calls,
    get_users,
)
from data_sources.sales import get_sales_data

from panels import key_metrics, conversion_attribution, kpi

In [2]:
first_calls_df = get_first_calls()
first_calls_df

,id,updatedAt,archived,associations,activity_type,hs_createdate,hs_guest_emails,hs_lastmodifieddate,hs_meeting_outcome,hs_meeting_start_time,meeting_title,hs_object_id,meeting_owner_id,contact_id,contact_email,conversion
date,,,,,,,,,,,,,,,,
2024-04-22 20:18:14.414000+00:00,27070610886,2025-03-06 10:23:20.307000+00:00,False,"{'contacts': [{'toObjectId': 9133471951, 'asso...",First Call,2024-04-22 20:18:14.414000+00:00,None,2025-03-06 10:23:20.307000+00:00,SCHEDULED,2024-04-23 10:30:00+00:00,Calendly: First call with LALIA,27070610886,362782088,9133471951,thairudorcas85@gmail.com,First call
2024-04-22 20:45:31.080000+00:00,27072654834,2025-03-06 10:23:20.307000+00:00,False,"{'contacts': [{'toObjectId': 9169088709, 'asso...",First Call,2024-04-22 20:45:31.080000+00:00,None,2025-03-06 10:23:20.307000+00:00,SCHEDULED,2024-04-25 09:00:00+00:00,Calendly: First call with LALIA,27072654834,<NA>,9169088709,laura.bortolini@hotmail.com,First call
2024-04-23 09:17:27.599000+00:00,27091118812,2025-03-06 10:23:20.307000+00:00,False,"{'contacts': [{'toObjectId': 9236133328, 'asso...",First Call,2024-04-23 09:17:27.599000+00:00,None,2025-03-06 10:23:20.307000+00:00,SCHEDULED,2024-04-29 10:00:00+00:00,Calendly: First call with LALIA,27091118812,<NA>,9236133328,elarkai96@gmail.com,First call
2024-04-23 13:24:39.820000+00:00,27101483452,2025-03-06 10:23:20.308000+00:00,False,"{'contacts': [{'toObjectId': 9284745672, 'asso...",First Call,2024-04-23 13:24:39.820000+00:00,None,2025-03-06 10:23:20.308000+00:00,SCHEDULED,2024-04-24 09:00:00+00:00,Calendly: First call with LALIA,27101483452,<NA>,9284745672,angellshamieh5@gmail.com,First call
2024-04-24 08:55:08.235000+00:00,27131671776,2025-03-06 10:23:20.308000+00:00,False,"{'contacts': [{'toObjectId': 9413510904, 'asso...",First Call,2024-04-24 08:55:08.235000+00:00,None,2025-03-06 10:23:20.308000+00:00,SCHEDULED,2024-04-26 09:30:00+00:00,Calendly: First call with LALIA,27131671776,<NA>,9413510904,munhungeimagna@gmail.com,First call
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-05 20:07:28.013000+00:00,155644200169,2025-03-18 09:09:01.486000+00:00,False,"{'contacts': [{'toObjectId': 218424846524, 'as...",First Call,2025-03-05 20:07:28.013000+00:00,None,2025-03-18 09:09:01.486000+00:00,COMPLETED,2025-03-06 09:30:00+00:00,Calendly: First call with LALIA,155644200169,1305224860,218424846524,petrinakrelle@gmail.com,First call
2025-03-05 21:25:28.757000+00:00,155591215300,2025-03-18 09:09:01.600000+00:00,False,"{'contacts': [{'toObjectId': 218485954764, 'as...",First Call,2025-03-05 21:25:28.757000+00:00,None,2025-03-18 09:09:01.600000+00:00,COMPLETED,2025-03-10 12:15:00+00:00,Calendly: First call with LALIA,155591215300,2114235671,218485954764,adaamirm@gmail.com,First call
2025-03-06 08:16:07.679000+00:00,155708190909,2025-03-18 09:08:59.423000+00:00,False,"{'contacts': [{'toObjectId': 11884314104, 'ass...",First Call,2025-03-06 08:16:07.679000+00:00,None,2025-03-18 09:08:59.423000+00:00,COMPLETED,2025-03-10 10:30:00+00:00,Calendly: First call with LALIA,155708190909,1305224860,11884314104,adubey0803@gmail.com,First call


In [3]:
@pn.cache(ttl=3600, to_disk=True)
def get_daily_data() -> pd.DataFrame:
    """Fetch all data sources and combine into a single dataframe with a datetime index.
    Returns:
        pd.DataFrame: A dataframe with a datetime index.
    """
    hs = get_hubspot_conversions(filters=None)
    sales = get_sales_data()
    fb = get_facebook_ads_campaign_metrics()
    ga = get_google_ads_campaign_metrics()
    lp = get_landing_page_report()
    data = pd.concat([hs, sales, fb, ga, lp]).infer_objects().convert_dtypes()
    # Ensure index is datetime
    data.index = pd.to_datetime(data.index)
    return data


data = get_daily_data()

[2025-04-22 10:33:13 - INFO] Retrieving deals from Hubspot
[2025-04-22 10:33:16 - INFO] Retrieved 857 deals from Hubspot in 2.92 seconds
[2025-04-22 10:33:16 - INFO] Retrieving first calls from Hubspot
[2025-04-22 10:37:26 - INFO] Retrieved 1397 first calls from Hubspot in 249.7 seconds
[2025-04-22 10:37:26 - INFO] Retrieving new calendly data since 2025-03-06 00:00:00
[2025-04-22 10:37:26 - INFO] Retrieving calendly data
[2025-04-22 10:39:04 - INFO] Retrieved 293 meetings from calendly in 97.6 seconds
[2025-04-22 10:39:04 - INFO] Retrieving sales data from Google Sheet
[2025-04-22 10:39:08 - INFO] Retrieved sales data in 4.04 seconds
[2025-04-22 10:39:08 - INFO] Retrieving new calendly data since 2025-03-06 00:00:00
[2025-04-22 10:39:08 - INFO] Retrieving calendly data
[2025-04-22 10:39:08 - INFO] Retrieved 293 calendly data from Hubspot in 0.03 seconds
[2025-04-22 10:39:08 - INFO] Retrieving Facebook Ads campaign metrics
[2025-04-22 10:39:47 - INFO] Retrieved Facebook Ads campaign me

In [13]:
print(
    data[
        [
            "campaign",
            "source",
            "medium",
            "content",
            "term",
            "first_call",
            "sales",
            "landing_page",
        ]
    ]
    .tail(10)
    .to_markdown()
)

| date                | campaign                       | source         | medium            | content                                              | term                                                                 | first_call   | sales   | landing_page   |
|:--------------------|:-------------------------------|:---------------|:------------------|:-----------------------------------------------------|:---------------------------------------------------------------------|:-------------|:--------|:---------------|
| 2025-04-08 00:00:00 | HM - TOF - ANGLE TESTING - ABO | ig             | Instagram_Stories | [T] TOF - AT - New Copies - Animated Video2 - New LP | [T] HM - TOF - AT - Value-Based Purchase LLA - AUTO & DC - New LP    | <NA>         | <NA>    | Hubspot2       |
| 2025-04-08 00:00:00 | HM - TOF - ANGLE TESTING - ABO | ig             | Instagram_Stories | [T] TOF - AT - New Copies - Rebecca2 (New LP)        | [T] HM - TOF - AT - Value-Based Purchase LLA - AUTO & DC - Mixed 

In [6]:
data.tail(50).to_markdown()

,meeting_owner_name,campaign,source,medium,content,term,first_call,placement_scheduled,verbal_agreement_after_first_call,sales,...,video_view_rate,engagement_rate,average_cpe,average_cpv,cost_per_conversion,hostname,landing_page,sessions,engaged_sessions,event_count
date,,,,,,,,,,,,,,,,,,,,,
2025-03-29,<NA>,HM - TOF - ANGLE TESTING - ABO,fb,Facebook_Mobile_Reels,[T] TOF - AT - New Copies - Review 2 - New LP,[T] HM - TOF - AT - Value-Based Purchase LLA -...,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,page.lalia-berlin.com,Hubspot2,1,0,4
2025-04-01,<NA>,HM - TOF - ANGLE TESTING - ABO,fb,Facebook_Mobile_Reels,[T] TOF - AT - New Copies - Review 2 - New LP,[T] HM - TOF - AT - Value-Based Purchase LLA -...,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,page.lalia-berlin.com,Hubspot2,1,1,7
2025-03-19,<NA>,HM - TOF - ANGLE TESTING - ABO,fb,Others,[T] TOF - AT - New Copies - Animated Video2 - ...,[T] HM - TOF - AT - Value-Based Purchase LLA -...,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,page.lalia-berlin.com,Hubspot2,1,0,4
2025-04-15,<NA>,HM - TOF - ANGLE TESTING - ABO,fb,Others,[T] TOF - AT - New Copies - Animated Video2 - ...,[T] HM - TOF - AT - Value-Based Purchase LLA -...,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,page.lalia-berlin.com,Hubspot2,1,0,3
2025-03-26,<NA>,HM - TOF - ANGLE TESTING - ABO,ig,Instagram_Feed,[T] TOF - AT - New Copies - American Girl (Vid...,[T] HM - TOF - AT - Value-Based Purchase LLA -...,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,page.lalia-berlin.com,Hubspot2,1,0,4
2025-03-28,<NA>,HM - TOF - ANGLE TESTING - ABO,ig,Instagram_Feed,[T] TOF - AT - New Copies - American Girl (Vid...,[T] HM - TOF - AT - Value-Based Purchase LLA -...,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,page.lalia-berlin.com,Hubspot2,1,1,8
2025-03-29,<NA>,HM - TOF - ANGLE TESTING - ABO,ig,Instagram_Feed,[T] TOF - AT - New Copies - American Girl (Vid...,[T] HM - TOF - AT - Value-Based Purchase LLA -...,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,page.lalia-berlin.com,Hubspot2,1,0,5
2025-03-13,<NA>,HM - TOF - ANGLE TESTING - ABO,ig,Instagram_Feed,[T] TOF - AT - New Copies - Animated Video2 - ...,[T] HM - TOF - AT - Value-Based Purchase LLA -...,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,page.lalia-berlin.com,Hubspot2,1,1,7
2025-03-15,<NA>,HM - TOF - ANGLE TESTING - ABO,ig,Instagram_Feed,[T] TOF - AT - New Copies - Animated Video2 - ...,[T] HM - TOF - AT - Value-Based Purchase LLA -...,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,page.lalia-berlin.com,Hubspot2,1,1,5


In [5]:
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

data

: 

: 

In [3]:
from hubspot_conversions import (
    get_first_calls,
    get_first_call_verbal_agreements,
    get_placement_calls,
)
# from sales import get_sales_data

first_calls_df = get_first_calls()
verbal_agreement_deals = get_first_call_verbal_agreements()
placement_deals = get_placement_calls()
# sales = get_sales_data()


[2025-03-18 10:22:58 - INFO] Retrieving first calls from Hubspot


[2025-03-18 10:26:50 - INFO] Retrieved 1397 first calls from Hubspot in 232.31 seconds
[2025-03-18 10:26:50 - INFO] Retrieving deals from Hubspot
[2025-03-18 10:26:53 - INFO] Retrieved 792 deals from Hubspot in 2.36 seconds


In [4]:
users = get_users()
users

first_name last_name                       created_at  archived  \
id                                                                            
362782088         Idan     Gillo 2024-04-22 07:19:13.445000+00:00     False   
637296518        Anton    Horeis 2024-04-22 07:23:12.520000+00:00     False   
642753864   Friederike  Knüpling 2024-04-30 08:50:55.239000+00:00     False   
1305224860      Office     Admin 2024-04-29 07:34:52.584000+00:00     False   
2114235671        Kyra    Laudor 2024-10-08 09:49:56.551000+00:00     False   

                                                        teams   user_id  \
id                                                                        
362782088                                                None  66278228   
637296518                                                None  66278310   
642753864                                                None  66582173   
1305224860  [{'name': 'Office', 'id': '158684582', 'primar...  66545026   
2114235671  [{'name': 'Office', 'id': '158684582', 'primar...  71878726   

                                  email                       updated_at  
id                                                                        
362782088        gillo@lalia-berlin.com 2024-05-07 12:20:58.857000+00:00  
637296518              anton@horeis.dev 2025-02-27 12:32:08.700000+00:00  
642753864   friederike@lalia-berlin.com 2024-11-26 13:19:31.428000+00:00  
1305224860      office@lalia-berlin.com 2025-03-17 06:36:12.937000+00:00  
2114235671        kyra@lalia-berlin.com 2025-03-17 08:33:21.817000+00:00

In [5]:
funnel_data = pd.concat(
    [first_calls_df, verbal_agreement_deals, placement_deals]
).dropna(subset=["contact_email", "meeting_owner_id"])
funnel_data["user"] = (
    users.loc[funnel_data.meeting_owner_id]["first_name"]
    + " "
    + users.loc[funnel_data.meeting_owner_id]["last_name"]
).values
funnel_data

id  \
date                                             
2024-04-22 20:18:14.414000+00:00   27070610886   
2024-04-24 14:59:42.383000+00:00   27147966908   
2024-04-24 17:19:57.860000+00:00   27152698064   
2024-04-25 10:47:31.705000+00:00   27175070704   
2024-04-26 10:05:38.538000+00:00   27211323343   
...                                        ...   
2025-03-13 10:15:03.543000+00:00  153458156765   
2025-03-17 10:13:42.423000+00:00  156869602503   
2025-03-17 10:23:51.034000+00:00  156939588839   
2025-03-17 10:39:58.968000+00:00  156925234383   
2025-03-17 11:23:16.999000+00:00  156943479024   

                                                        updatedAt  archived  \
date                                                                          
2024-04-22 20:18:14.414000+00:00 2025-03-06 10:23:20.307000+00:00     False   
2024-04-24 14:59:42.383000+00:00 2025-03-06 10:23:20.308000+00:00     False   
2024-04-24 17:19:57.860000+00:00 2025-03-06 10:23:20.308000+00:00     False   
2024-04-25 10:47:31.705000+00:00 2025-03-06 10:23:20.307000+00:00     False   
2024-04-26 10:05:38.538000+00:00 2025-03-06 10:23:20.307000+00:00     False   
...                                                           ...       ...   
2025-03-13 10:15:03.543000+00:00 2025-03-18 09:15:02.996000+00:00     False   
2025-03-17 10:13:42.423000+00:00 2025-03-17 11:14:42.930000+00:00     False   
2025-03-17 10:23:51.034000+00:00 2025-03-17 11:24:51.954000+00:00     False   
2025-03-17 10:39:58.968000+00:00 2025-03-17 11:41:00.520000+00:00     False   
2025-03-17 11:23:16.999000+00:00 2025-03-17 12:24:18.026000+00:00     False   

                                                                       associations  \
date                                                                                  
2024-04-22 20:18:14.414000+00:00  {'contacts': [{'toObjectId': 9133471951, 'asso...   
2024-04-24 14:59:42.383000+00:00  {'contacts': [{'toObjectId': 9503657917, 'asso...   
2024-04-24 17:19:57.860000+00:00  {'contacts': [{'toObjectId': 9530925514, 'asso...   
2024-04-25 10:47:31.705000+00:00  {'contacts': [{'toObjectId': 9647943926, 'asso...   
2024-04-26 10:05:38.538000+00:00  {'contacts': [{'toObjectId': 9806212798, 'asso...   
...                                                                             ...   
2025-03-13 10:15:03.543000+00:00                                                NaN   
2025-03-17 10:13:42.423000+00:00                                                NaN   
2025-03-17 10:23:51.034000+00:00                                                NaN   
2025-03-17 10:39:58.968000+00:00                                                NaN   
2025-03-17 11:23:16.999000+00:00                                                NaN   

                                 activity_type  \
date                                             
2024-04-22 20:18:14.414000+00:00    First Call   
2024-04-24 14:59:42.383000+00:00    First Call   
2024-04-24 17:19:57.860000+00:00    First Call   
2024-04-25 10:47:31.705000+00:00    First Call   
2024-04-26 10:05:38.538000+00:00    First Call   
...                                        ...   
2025-03-13 10:15:03.543000+00:00          <NA>   
2025-03-17 10:13:42.423000+00:00          <NA>   
2025-03-17 10:23:51.034000+00:00          <NA>   
2025-03-17 10:39:58.968000+00:00          <NA>   
2025-03-17 11:23:16.999000+00:00          <NA>   

                                                    hs_createdate  \
date                                                                
2024-04-22 20:18:14.414000+00:00 2024-04-22 20:18:14.414000+00:00   
2024-04-24 14:59:42.383000+00:00 2024-04-24 14:59:42.383000+00:00   
2024-04-24 17:19:57.860000+00:00 2024-04-24 17:19:57.860000+00:00   
2024-04-25 10:47:31.705000+00:00 2024-04-25 10:47:31.705000+00:00   
2024-04-26 10:05:38.538000+00:00 2024-04-26 10:05:38.538000+00:00   
...                                                           ...   
2025-03-13 10:15:03.543000+00:

In [6]:
import plotly.graph_objects as go
import urllib, json

url = "https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json"
response = urllib.request.urlopen(url)
data = json.loads(response.read())

# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity
data["data"][0]["node"]["color"] = [
    "rgba(255,0,255, 0.8)" if color == "magenta" else color
    for color in data["data"][0]["node"]["color"]
]
data["data"][0]["link"]["color"] = [
    data["data"][0]["node"]["color"][src].replace("0.8", str(opacity))
    for src in data["data"][0]["link"]["source"]
]

fig = go.Figure(
    data=[
        go.Sankey(
            valueformat=".0f",
            valuesuffix="TWh",
            # Define nodes
            node=dict(
                pad=15,
                thickness=15,
                line=dict(color="black", width=0.5),
                label=data["data"][0]["node"]["label"],
                color=data["data"][0]["node"]["color"],
            ),
            # Add links
            link=dict(
                source=data["data"][0]["link"]["source"],
                target=data["data"][0]["link"]["target"],
                value=data["data"][0]["link"]["value"],
                label=data["data"][0]["link"]["label"],
                color=data["data"][0]["link"]["color"],
            ),
        )
    ]
)

fig.update_layout(
    title_text="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
    font_size=10,
)
fig.show()

In [29]:
conversion_to_label = {
    "First call": "First Call scheduled",
    "Verbal agreement after first call": "First Call Verbal Agreement",
    "Placement scheduled": "Placement scheduled",
    "Verbal agreement after placement call": "Placement Verbal Agreement",
    "Sale": "Sale",
}

labels = [
    "First Call",
    "First Call Verbal Agreement",
    "Placement Call",
    "Placement Verbal Agreement",
    "Sale",
]

# Define allowed transitions in the funnel
transitions = [
    ("First Call", "First Call Verbal Agreement"),
    ("First Call", "Placement Call"),
    ("First Call Verbal Agreement", "Sale"),
    ("Placement Call", "Placement Verbal Agreement"),
    ("Placement Call", "Sale"),
    ("Placement Verbal Agreement", "Sale"),
]

# Dynamically create sources and targets arrays
sources = []
targets = []
for source_label, target_label in transitions:
    sources.append(labels.index(source_label))
    targets.append(labels.index(target_label))


def get_index(from_conversion, to_conversion):
    """Get the index in the values array for a specific transition between stages"""
    try:
        from_idx = labels.index(from_conversion)
        to_idx = labels.index(to_conversion)

        for idx, (src, tgt) in enumerate(zip(sources, targets)):
            if src == from_idx and tgt == to_idx:
                return idx

        return -1  # Return -1 if not found
    except ValueError:
        return -1  # Return -1 if label not found


def compute_sankey(group):
    # Create a list to store individual links
    links = []  # Each link will be a tuple of (source, target, value, user)

    conversions = group.conversion.values
    if "First call" not in conversions:
        return links

    # Process each transition based on the user who handled it
    if "Verbal agreement after first call" in conversions:
        # Get the specific user who handled this conversion
        user = group.user.loc[
            group.conversion == "Verbal agreement after first call"
        ].iloc[0]
        source_idx = labels.index("First Call")
        target_idx = labels.index("First Call Verbal Agreement")
        links.append((source_idx, target_idx, 1, user))

        if "closedwon" in group.dealstage.values:
            source_idx = labels.index("First Call Verbal Agreement")
            target_idx = labels.index("Sale")
            links.append((source_idx, target_idx, 1, user))

    if "Placement scheduled" in conversions:
        # Get the user who scheduled the first call
        fc_user = group.user.loc[group.conversion == "First call"].iloc[0]
        source_idx = labels.index("First Call")
        target_idx = labels.index("Placement Call")
        links.append((source_idx, target_idx, 1, fc_user))

        # Get the user who scheduled the placement
        if any(group.conversion == "Placement scheduled"):
            placement_user = group.user.loc[
                group.conversion == "Placement scheduled"
            ].iloc[0]

            if "true" in group.verbal_agreement.values:
                source_idx = labels.index("Placement Call")
                target_idx = labels.index("Placement Verbal Agreement")
                links.append((source_idx, target_idx, 1, placement_user))

                if "closedwon" in group.dealstage.values:
                    source_idx = labels.index("Placement Verbal Agreement")
                    target_idx = labels.index("Sale")
                    links.append((source_idx, target_idx, 1, placement_user))

            elif "closedwon" in group.dealstage.values:
                source_idx = labels.index("Placement Call")
                target_idx = labels.index("Sale")
                links.append((source_idx, target_idx, 1, placement_user))

    return links


# Process data to get individual links
all_links = []
for email, group_data in funnel_data.groupby("contact_email"):
    links = compute_sankey(
        group_data[["conversion", "dealstage", "verbal_agreement", "user"]]
    )
    all_links.extend(links)


# Create a dynamic color palette for users
def generate_color_palette(users):
    """Generate a color palette for users with consistent colors"""
    import colorsys
    import numpy as np

    # Get unique users
    unique_users = sorted(list(set(users)))
    n = len(unique_users)

    # Generate colors using HSV color space to ensure they're visually distinct
    colors = {}
    for i, user in enumerate(unique_users):
        # Generate evenly spaced hues
        hue = i / n
        # Use high saturation and value for vibrant colors
        saturation = 0.7
        value = 0.9

        # Convert HSV to RGB
        r, g, b = colorsys.hsv_to_rgb(hue, saturation, value)

        # Convert to rgba string with opacity
        colors[user] = f"rgba({int(r * 255)}, {int(g * 255)}, {int(b * 255)}, 0.8)"

    return colors


# Extract all users from links
all_users = [link[3] for link in all_links]
user_colors = generate_color_palette(all_users)

# Group links by source, target, and user
from collections import defaultdict

link_dict = defaultdict(int)
for source, target, value, user in all_links:
    link_dict[(source, target, user)] += value

# Prepare data for Sankey diagram
sources = []
targets = []
values = []
colors = []
user_labels = []

for (source, target, user), value in link_dict.items():
    sources.append(source)
    targets.append(target)
    values.append(value)
    colors.append(user_colors[user])
    user_labels.append(user)

# Apply the same color scheme to nodes (based on position in funnel)
node_colors = [f"rgba(150, 150, 150, 0.8)"] * len(labels)  # Neutral color for nodes

# Create the Sankey diagram
fig = go.Figure(
    data=[
        go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=labels,
                color=node_colors,
            ),
            link=dict(
                source=sources,
                target=targets,
                value=values,
                label=user_labels,  # Show user name on hover
                color=colors,  # Color by user
            ),
        )
    ]
)

# Create a custom legend for user colors
legend_y_start = 1.0
legend_items = []

# Add title and descriptions
fig.update_layout(
    title_text="Conversion Funnel by User",
    font_size=10,
    height=600,
    width=1000,
    margin=dict(t=100, l=50, r=250),  # Extra right margin for legend
)

# Add a custom legend using annotations
for i, (user, color) in enumerate(sorted(user_colors.items())):
    y_pos = legend_y_start - (i * 0.06)

    # Add colored rectangle
    fig.add_shape(
        type="rect",
        xref="paper",
        yref="paper",
        x0=1.02,
        y0=y_pos - 0.02,
        x1=1.05,
        y1=y_pos + 0.02,
        fillcolor=color,
        line=dict(color="black", width=1),
    )

    # Add user name
    fig.add_annotation(
        xref="paper",
        yref="paper",
        x=1.06,
        y=y_pos + 0.01,
        text=user,
        showarrow=False,
        xanchor="left",
    )

# Add legend title
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=1,
    y=legend_y_start + 0.1,
    text="User Legend",
    showarrow=False,
    xanchor="left",
    font=dict(size=16, color="black", family="Arial"),
)

fig.show()


In [17]:
"Kyra Laudor" in group.loc[funnel_data.conversion == "First call"].user

user
Office Admin           844
Kyra Laudor            296
Idan Gillo             145
Friederike Knüpling    102
Name: count, dtype: int64

In [15]:
funnel_data.verbal_agreement.unique()

<StringArray>
[<NA>, 'true', 'false']
Length: 3, dtype: string